In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import os
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
excel_file=pd.ExcelFile("./All SOTF Opens to Date_8.27.xlsx")
excel_file.sheet_names

['Sheet2']

In [3]:
df=excel_file.parse("Sheet2",dtype=str,skiprows=2)
df_2018=df[['Store ID','City']].drop_duplicates()
df_2019=df[['Store ID.1','City.1']].rename(columns={"Store ID.1":"Store ID","City.1":"City"}).drop_duplicates()

df_2018=df_2018[pd.notnull(df_2018['Store ID'])]
df_2019=df_2019[pd.notnull(df_2019['Store ID'])]
df_2018=df_2018[df_2018['Store ID']!="nan"]
df_2019=df_2019[df_2019['Store ID']!="nan"]

print(df_2018.shape,df_2018['Store ID'].nunique())
print(df_2019.shape,df_2019['Store ID'].nunique())
df_2019['SOTF_year']=2019
df_2018['SOTF_year']=2018
df_output=df_2019.append(df_2018)
print(df_output.shape,df_output['Store ID'].nunique())
df_output=df_output.drop_duplicates("Store ID")

(177, 2) 173
(128, 2) 128
(305, 3) 299


In [4]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190801-135940-752.txt",sep="|",dtype=str)
store_list=store_list[['location_id','latitude_meas','longitude_meas','zip_cd']]
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list=store_list.rename(columns={"zip_cd":"store_zip"})

df_output=pd.merge(df_output,store_list,left_on="Store ID",right_on="location_id",how="left")
del df_output['location_id']

In [5]:
df_store_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
df_store_zips.sheet_names


['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [6]:
df_old_store_zips=df_store_zips.parse("view_by_store",dtype=str)
df_old_store_zips=df_old_store_zips[['location_id','all_trans_P_zips','all_trans_S_zips','zips_in_10']]
df_old_store_zips['all_trans_P_zips']=df_old_store_zips['all_trans_P_zips'].apply(lambda x: eval(x))
df_old_store_zips['all_trans_S_zips']=df_old_store_zips['all_trans_S_zips'].apply(lambda x: eval(x))
df_old_store_zips['zips_in_10']=df_old_store_zips['zips_in_10'].apply(lambda x: eval(x))
df_old_store_zips=df_old_store_zips[df_old_store_zips['location_id'].isin(df_output['Store ID'].tolist())]
df_old_store_zips=df_old_store_zips.reset_index()
del df_old_store_zips['index']

In [7]:
df_output_old_stores=pd.DataFrame()
for ind, row in df_old_store_zips.iterrows():
    if row['all_trans_P_zips']:
        df_P=pd.DataFrame({"Store ID":[row['location_id']]*len(row['all_trans_P_zips']),"zip_cd":row['all_trans_P_zips']},
                          index=["P"]*len(row['all_trans_P_zips']))
    else:
        df_P=pd.DataFrame()
        
    if row['all_trans_S_zips']:
        df_S=pd.DataFrame({"Store ID":[row['location_id']]*len(row['all_trans_S_zips']),"zip_cd":row['all_trans_S_zips']},
                          index=["S"]*len(row['all_trans_S_zips']))
    else:
        df_S=pd.DataFrame()
        
    if row['zips_in_10']:
        df_10=pd.DataFrame({"Store ID":[row['location_id']]*len(row['zips_in_10']),"zip_cd":row['zips_in_10']},
                          index=["zip_10"]*len(row['zips_in_10']))
    else:
        df_10=pd.DataFrame()
        
    df_store=pd.concat([df_P,df_S,df_10]).reset_index().rename(columns={"index":"trans_label"})
    df_output_old_stores=df_output_old_stores.append(df_store)

In [8]:
df_output_old_stores=pd.merge(df_output,df_output_old_stores,on="Store ID",how="left")
df_output_old_stores=df_output_old_stores[pd.notnull(df_output_old_stores['trans_label'])]
df_output_old_stores.head(2)

,Store ID,City,SOTF_year,latitude_meas,longitude_meas,store_zip,trans_label,zip_cd
0,4687,"BAY SHORE, NY",2019,40.739273,-73.236436,11706,P,11706
1,4687,"BAY SHORE, NY",2019,40.739273,-73.236436,11706,P,11717


In [9]:
df_output_old_stores=df_output_old_stores.reset_index()
del df_output_old_stores['index']
df_output_old_stores['dist_miles']=np.nan
for ind, row in df_output_old_stores.iterrows():
    if row['zip_cd'] in zip_centers.keys():
        dist=haversine((row['latitude_meas'],row['longitude_meas']),zip_centers[row['zip_cd']],miles=True)
    else:
        dist=np.nan
    df_output_old_stores.loc[ind,"dist_miles"]=dist

In [10]:
df_output_old_stores.head(2)

,Store ID,City,SOTF_year,latitude_meas,longitude_meas,store_zip,trans_label,zip_cd,dist_miles
0,4687,"BAY SHORE, NY",2019,40.739273,-73.236436,11706,P,11706,0.787626
1,4687,"BAY SHORE, NY",2019,40.739273,-73.236436,11706,P,11717,3.156137


In [11]:
df_output_old_stores['trans_label'].unique()

array(['P', 'S', 'zip_10'], dtype=object)

In [12]:
new_store_lat_lng_dict={}
# 5406 	2525 DAWSON ROAD	KMART CENTER	ALBANY	GA	31707-2315
new_store_lat_lng_dict.update({"5406":{"Google_Address":"2525 Dawson Rd, Albany, GA 31707","LatLng":[31.612023,-84.2123335]}})

# 5403 	3550 NORTHWEST FEDERAL HIGHWAY	MARKETPLACE SQUARE	JENSEN BEACH	FL	34957-4406
new_store_lat_lng_dict.update({"5403":{"Google_Address":"3550 NW Federal Hwy Jensen Beach, FL 34957","LatLng":[27.2463725,-80.2738352]}})

# 5407 	14154 EAST WADE HAMPTON BLVD	COLLINS CENTER	GREER	SC	29651-1554
new_store_lat_lng_dict.update({"5407":{"Google_Address":"14154 E Wade Hampton Blvd Greer, SC 29651","LatLng":[34.9489194,-82.2119013]}})

# 4704 	4815 EAST FREEWAY		BAYTOWN	TX	77521-8369
new_store_lat_lng_dict.update({"4704":{"Google_Address":"4815 East Fwy Baytown, TX 77521","LatLng":[29.8051762,-94.9864945]}})

# 5404 	142 EXECUTIVE DRIVE	TOYS R US CENTER	DANVILLE	VA	24541-4100
new_store_lat_lng_dict.update({"5404":{"Google_Address":"142 Executive Dr, Danville, VA 24541","LatLng":[36.5932645,-79.4309145]}})

# 5388 	4610 FREDERICA STREET	WESLEYAN PARK PLAZA	OWENSBORO	KY	42301-7496
new_store_lat_lng_dict.update({"5388":{"Google_Address":"4610 Frederica St Owensboro, KY 42301","LatLng":[37.7255617,-87.1261072]}})

# 4708 	6727 EVERGREEN WAY	ALBERTSONS SC	EVERETT	WA	98203
new_store_lat_lng_dict.update({"4708":{"Google_Address":"6727 Evergreen Way, Everett, WA 98203","LatLng":[47.937259,-122.2175977]}})


In [13]:
new_store=df_output[~df_output['Store ID'].isin(df_output_old_stores['Store ID'].tolist())]
new_store=new_store.reset_index()
del new_store['index']
for ind, row in new_store.iterrows():
    if row['Store ID'] in new_store_lat_lng_dict.keys():
        new_store.loc[ind,"latitude_meas"]=new_store_lat_lng_dict[row['Store ID']]['LatLng'][0]
        new_store.loc[ind,"longitude_meas"]=new_store_lat_lng_dict[row['Store ID']]['LatLng'][1]
        new_store.loc[ind,"store_zip"]=new_store_lat_lng_dict[row['Store ID']]['Google_Address'][-5:]
        
new_store       

,Store ID,City,SOTF_year,latitude_meas,longitude_meas,store_zip
0,4679,"OKEMOS, MI",2019,42.723314,-84.424187,48864
1,4698,"VIENNA, WV",2019,39.300077,-81.548860,26105
2,5394,"BOCA RATON, FL",2019,26.353521,-80.205267,33428
3,5406,"ALBANY, GA",2019,31.612023,-84.212333,31707
4,4692,"BELLINGHAM, WA",2019,48.775269,-122.495657,98225
5,5403,"JENSEN BEACH, FL",2019,27.246372,-80.273835,34957
6,5407,"GREER, SC",2019,34.948919,-82.211901,29651
7,4704,"BAYTOWN, TX",2019,29.805176,-94.986495,77521
8,5404,"DANVILLE, VA",2019,36.593264,-79.430915,24541
9,5388,"OWENSBORO, KY",2019,37.725562,-87.126107,42301


In [14]:
df_zips_in_10_new_stores=pd.DataFrame()
for ind, row in new_store.iterrows():
    store_lat_long=(row['latitude_meas'],row['longitude_meas'])
    store_id=row['Store ID']
    for zip_cd in zip_centers.keys():
        dist=haversine(store_lat_long,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df_zips_in_10_new_stores=df_zips_in_10_new_stores.append(pd.DataFrame({"zip_cd":zip_cd,
                                                                                   "trans_label":"zip_10",
                                                                                   "dist_miles":dist},
                                                                                  index=[store_id]))
df_zips_in_10_new_stores=df_zips_in_10_new_stores.reset_index().rename(columns={"index":"Store ID"})
df_output_new_stores=pd.merge(new_store,df_zips_in_10_new_stores,on="Store ID",how="left")
df_output_new_stores=df_output_new_stores[df_output_old_stores.columns.tolist()]

In [15]:
df_output=df_output_old_stores.append(df_output_new_stores)
df_output=df_output[((df_output['dist_miles']<=20) | (df_output['trans_label']=="P"))]

In [16]:
df_unique_zips=df_output[['zip_cd','trans_label']].sort_values(["zip_cd",'trans_label']).drop_duplicates("zip_cd")

In [17]:
writer=pd.ExcelWriter("./All SOTF Opens to Date_8.27_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zips.to_excel(writer,"unique_zips",index=False)  
df_output.to_excel(writer,"zip_by_store",index=False)                  
writer.save()